# Data Preparation for Online Retail Dataset

## 1. Load the Data

In [24]:

import pandas as pd

# Load dataset
snapdeal = pd.read_csv(
    r'C:\Users\venne\Documents\Mentormind\Snapdeal_project\Online Retail.csv',
    encoding="latin1"
)
# Display shape and first rows
print("Shape:", snapdeal.shape)
snapdeal.head()

Shape: (541909, 8)


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850.0,United Kingdom


## 2. Understand the Data

In [25]:

# Basic info
snapdeal.info()

# Statistical summary
snapdeal.describe(include="all")

# Check missing values
snapdeal.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   InvoiceNo    541909 non-null  object 
 1   StockCode    541909 non-null  object 
 2   Description  540455 non-null  object 
 3   Quantity     541909 non-null  int64  
 4   InvoiceDate  541909 non-null  object 
 5   UnitPrice    541909 non-null  float64
 6   CustomerID   406829 non-null  float64
 7   Country      541909 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

## 3. Clean the Data

In [26]:
# Drop duplicates
data = data.drop_duplicates()
# Convert InvoiceDate to datetime
data['InvoiceDate'] = pd.to_datetime(data['InvoiceDate'], format='%d/%m/%Y', errors='coerce')
# Handle missing values
# Drop rows where CustomerID is missing (important for customer-level analysis)
data = data.dropna(subset=['CustomerID'])
# Fill missing descriptions with 'Unknown'
data['Description'] = data['Description'].fillna('Unknown')
# Convert CustomerID to integer
data['CustomerID'] = data['CustomerID'].astype(int)
data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 392691 entries, 0 to 541908
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   InvoiceNo     392691 non-null  object        
 1   StockCode     392691 non-null  object        
 2   Description   392691 non-null  object        
 3   Quantity      392691 non-null  float64       
 4   InvoiceDate   0 non-null       datetime64[ns]
 5   UnitPrice     392691 non-null  float64       
 6   CustomerID    392691 non-null  int64         
 7   Country       392691 non-null  int64         
 8   TotalPrice    392691 non-null  float64       
 9   InvoiceYear   0 non-null       float64       
 10  InvoiceMonth  0 non-null       float64       
 11  InvoiceDay    0 non-null       float64       
 12  InvoiceHour   0 non-null       float64       
dtypes: datetime64[ns](1), float64(7), int64(2), object(3)
memory usage: 41.9+ MB


## 4. Handle Anomalies

In [27]:
# Remove rows with negative or zero Quantity or UnitPrice
data = data[(data['Quantity'] > 0) & (data['UnitPrice'] > 0)]

# Check cleaned dataset
data.describe()

,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice,InvoiceYear,InvoiceMonth,InvoiceDay,InvoiceHour
count,5613.000000,0,5613.000000,5613.000000,5613.000000,5613.000000,0.0,0.0,0.0,0.0
mean,0.202225,NaT,0.099659,15134.784785,30.914662,0.015677,NaN,NaN,NaN,NaN
min,0.004877,NaT,0.001083,12356.000000,0.000000,0.000027,NaN,NaN,NaN,NaN
25%,0.038119,NaT,0.028059,13767.000000,35.000000,0.001691,NaN,NaN,NaN,NaN
50%,0.060281,NaT,0.050539,14911.000000,35.000000,0.003873,NaN,NaN,NaN,NaN
75%,0.193250,NaT,0.123824,16701.000000,35.000000,0.009875,NaN,NaN,NaN,NaN
max,7.750337,NaT,29.061184,18272.000000,35.000000,7.548201,NaN,NaN,NaN,NaN
std,0.430243,NaN,0.403997,1790.352136,9.019818,0.108738,NaN,NaN,NaN,NaN


## 5. Transform the Data

In [28]:

from sklearn.preprocessing import StandardScaler, LabelEncoder

# Example: Normalize UnitPrice and Quantity
scaler = StandardScaler()
data[['Quantity', 'UnitPrice']] = scaler.fit_transform(data[['Quantity', 'UnitPrice']])

# Encode Country
encoder = LabelEncoder()
data['Country'] = encoder.fit_transform(data['Country'])

data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice,InvoiceYear,InvoiceMonth,InvoiceDay,InvoiceHour
26,536370,22728,ALARM CLOCK BAKELIKE PINK,-0.329944,NaT,-0.177244,12583,9,0.001691,NaN,NaN,NaN,NaN
27,536370,22727,ALARM CLOCK BAKELIKE RED,-0.329944,NaT,-0.177244,12583,9,0.001691,NaN,NaN,NaN,NaN
65,536374,21258,VICTORIAN SEWING BOX LARGE,-0.226917,NaT,0.624105,15100,25,0.036797,NaN,NaN,NaN,NaN
82,536376,22114,HOT WATER BOTTLE TEA AND SYMPATHY,-0.020861,NaT,-0.210634,15291,25,0.002816,NaN,NaN,NaN,NaN
106,536381,22139,RETROSPOT TEA SET CERAMIC 11 PC,-0.342823,NaT,-0.121595,15311,25,0.002767,NaN,NaN,NaN,NaN


## 6. Feature Engineering

In [29]:

# Create new feature: Total Price
data['TotalPrice'] = data['Quantity'] * data['UnitPrice']

# Example: Extract date features
data['InvoiceYear'] = data['InvoiceDate'].dt.year
data['InvoiceMonth'] = data['InvoiceDate'].dt.month
data['InvoiceDay'] = data['InvoiceDate'].dt.day
data['InvoiceHour'] = data['InvoiceDate'].dt.hour

data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice,InvoiceYear,InvoiceMonth,InvoiceDay,InvoiceHour
26,536370,22728,ALARM CLOCK BAKELIKE PINK,-0.329944,NaT,-0.177244,12583,9,0.058481,NaN,NaN,NaN,NaN
27,536370,22727,ALARM CLOCK BAKELIKE RED,-0.329944,NaT,-0.177244,12583,9,0.058481,NaN,NaN,NaN,NaN
65,536374,21258,VICTORIAN SEWING BOX LARGE,-0.226917,NaT,0.624105,15100,25,-0.141620,NaN,NaN,NaN,NaN
82,536376,22114,HOT WATER BOTTLE TEA AND SYMPATHY,-0.020861,NaT,-0.210634,15291,25,0.004394,NaN,NaN,NaN,NaN
106,536381,22139,RETROSPOT TEA SET CERAMIC 11 PC,-0.342823,NaT,-0.121595,15311,25,0.041685,NaN,NaN,NaN,NaN


## 7. Export Cleaned Data

In [31]:
# Save the cleaned dataset to CSV
cleaned_file_path = r"C:\Users\venne\Documents\Mentormind\Snapdeal_project\cleaned_online_retail.csv"
data.to_csv(cleaned_file_path, index=False)
print(f"Cleaned dataset saved as {cleaned_file_path}")

Cleaned dataset saved as C:\Users\venne\Documents\Mentormind\Snapdeal_project\cleaned_online_retail.csv
